# 戦闘から得られたAPI履歴から、必要なデータを抽出するプロセスを構築する

In [1]:
import json
import os

In [2]:
# 対象のrootディレクトリ
root_dir = os.getcwd() + '/2025-01-30 10:31:15.605343'

## api_req_map/start 海域出撃開始時

In [ ]:
with open(root_dir + '/2025-01-30 10:34:23.688628_api_req_map-start.json') as f:
    start_response = f.read()
    start_response = json.loads(start_response).get("api_data")
    

### 羅針盤が表示されるか確認

In [4]:
print("羅針盤が表示される" if start_response.get("api_rashin_flg") else "羅針盤が表示されない")

羅針盤が表示されない


### 次に訪れるセル

In [5]:
next_cell_id = start_response.get("api_no")
cell_data = start_response.get("api_cell_data")

for d in cell_data:
    if d.get("api_no") == next_cell_id:
        next_cell_data = d
        break

print(f"次に訪れるセルのID: {next_cell_id}")
print(f"次に訪れるセルの情報: {next_cell_data}")

次に訪れるセルのID: 1
次に訪れるセルの情報: {'api_id': 3002, 'api_no': 1, 'api_color_no': 4, 'api_passed': 1}


### 次のマスで起こるイベント

In [6]:
event_id = start_response.get("api_event_id")
event_mapping = {
    0: "初期位置",
    1: "イベントなし",
    2: "資源獲得",
    3: "渦潮",
    4: "通常戦闘",
    5: "ボス戦闘",
    6: "気のせいだった",
    7: "航空戦or航空偵察",
    8: "船団護衛成功",
    9: "揚陸地点",
    10: "泊地"
}

event_description = event_mapping.get(event_id, "未知のイベント")
print(f"イベントID: {event_id}, イベント: {event_description}")

イベントID: 4, イベント: 通常戦闘


## api_req_sortie/battle 戦闘開始

In [7]:
with open(root_dir + "/2025-01-30 10:34:52.025271_api_req_sortie-battle.json") as f:
    battle_response = f.read()
    battle_response = json.loads(battle_response).get("api_data")

### 戦闘の情報

In [8]:
opening_taisen_flag = battle_response.get("api_opening_taik_flag")
opening_flag = battle_response.get("api_opening_flag")
hourai_flag = battle_response.get("api_hourai_flag")

print("先制対潜が発生する" if opening_taisen_flag else "先制対潜が発生しない")
print("先制雷撃が発生する" if opening_flag else "先制雷撃が発生しない")
print("砲撃戦１巡目が発生する" if hourai_flag[0] else "砲撃戦１巡目が発生しない")
print("砲撃戦２巡目が発生する" if hourai_flag[1] else "砲撃戦２巡目が発生しない")
print("砲撃戦３巡目が発生する" if hourai_flag[2] else "砲撃戦３巡目が発生しない")
print("雷撃戦が発生する" if hourai_flag[3] else "雷撃戦が発生しない")

先制対潜が発生しない
先制雷撃が発生しない
砲撃戦１巡目が発生する
砲撃戦２巡目が発生しない
砲撃戦３巡目が発生しない
雷撃戦が発生しない


### 敵味方のHPを取得

In [9]:
friend_now_hp_list = battle_response.get("api_f_nowhps")
friend_max_hp_list = battle_response.get("api_f_maxhps")
enemy_now_hp_list = battle_response.get("api_e_nowhps")
enemy_max_hp_list = battle_response.get("api_e_maxhps")

for i, friend_now_hp in enumerate(friend_now_hp_list):
    print(f"味方{i+1}隻目のHP: {friend_now_hp}/{friend_max_hp_list[i]}")

for i, enemy_now_hp in enumerate(enemy_now_hp_list):
    print(f"敵{i+1}隻目のHP: {enemy_now_hp}/{enemy_max_hp_list[i]}")

味方1隻目のHP: 15/15
味方2隻目のHP: 13/13
味方3隻目のHP: 15/15
味方4隻目のHP: 15/15
味方5隻目のHP: 13/13
味方6隻目のHP: 16/16
敵1隻目のHP: 20/20


### 敵味方のダメージを算出

In [10]:

total_friend_damage_list = [0] * 7
total_enemy_damage_list = [0] * 7

if opening_taisen_flag:
    # TODO 先制対潜の情報を取得する
    pass
else:
    print("<先制対潜は発生しませんでした>")

if opening_flag:
    # TODO 先制雷撃の情報を取得する
    pass
else:
    print("<先制雷撃は発生しませんでした>")

# 砲撃戦の情報を取得する
for i in range(3):
    flag = hourai_flag[i]
    if not flag:
        break
    
    print("<砲撃戦" + str(i+1) + "巡目>")
    
    hougeki_data = battle_response.get(f"api_hougeki{i+1}")
    # 行動陣営フラグ 0=味方, 1=敵
    at_e_flag_list = hougeki_data.get("api_at_eflag")
    df_list = hougeki_data.get("api_df_list")
    damage_list = hougeki_data.get("api_damage")
    for i, at_e_flag in enumerate(at_e_flag_list):
        index = df_list[i][0]
        damage = damage_list[i][0]
        is_protected = damage % 1 == 0.1
        damage //= 1
        
        # ダメージを記録
        if at_e_flag:
            print(f"味方の{index + 1}隻目に{damage}ダメージ{"(旗艦を庇った)" if is_protected else ""}")
            total_friend_damage_list[index] += damage
        else:
            print(f"敵の{index + 1}隻目に{damage}ダメージ{"(旗艦を庇った)" if is_protected else ""}")
            total_enemy_damage_list[index] += damage

# 雷撃戦の情報を取得する
if hourai_flag[3]:
    raigeki = battle_response.get("api_raigeki")
    fdam = raigeki.get("api_fdam")
    edam = raigeki.get("api_edam")

    for i, f in enumerate(fdam):
        print(f"味方の{i+1}隻目に{f}ダメージ")
        total_friend_damage_list[i] += f

    for i, e in enumerate(edam):
        print(f"敵の{i+1}隻目に{e}ダメージ")
        total_enemy_damage_list[i] += e
else:
    print("<雷撃戦は発生しませんでした>")

print("<トータル>")
print(f"味方のダメージ合計: {total_friend_damage_list}")
print(f"敵のダメージ合計: {total_enemy_damage_list}")

<先制対潜は発生しませんでした>
<先制雷撃は発生しませんでした>
<砲撃戦1巡目>
敵の1隻目に12ダメージ
味方の4隻目に1ダメージ
敵の1隻目に24ダメージ
<雷撃戦は発生しませんでした>
<トータル>
味方のダメージ合計: [0, 0, 0, 1, 0, 0, 0]
敵のダメージ合計: [36, 0, 0, 0, 0, 0, 0]


### 残存HPを算出 大破かどうかを算出

In [11]:
friend_rest_hp_list = [now - damage for now, damage in zip(friend_now_hp_list, total_friend_damage_list)]
enemy_rest_hp_list = [now - damage for now, damage in zip(enemy_now_hp_list, total_enemy_damage_list)]

for i, (rest, max) in enumerate(zip(friend_rest_hp_list, friend_max_hp_list)):
    huge_damage = rest <= max // 4
    print(f"味方{i+1}隻目の残りHP: {rest}/{max} {"(大破)" if huge_damage else ""}")

for i, (rest, max) in enumerate(zip(enemy_rest_hp_list, enemy_max_hp_list)):
    print(f"敵{i+1}隻目の残りHP: {rest}/{max}")

味方1隻目の残りHP: 15/15 
味方2隻目の残りHP: 13/13 
味方3隻目の残りHP: 15/15 
味方4隻目の残りHP: 14/15 
味方5隻目の残りHP: 13/13 
味方6隻目の残りHP: 16/16 
敵1隻目の残りHP: -16/20


### 夜戦を選択可能かどうか確認

In [12]:
can_midnight_battle = any([hp > 0 for hp in enemy_rest_hp_list])
print("夜戦を行える" if can_midnight_battle else "夜戦を行えない")

夜戦を行えない


## api_req_sortie/battleresult 戦闘結果

In [13]:
with open(root_dir + "/2025-01-30 10:35:07.124012_api_req_sortie-battleresult.json") as f:
    battle_result = f.read()
    battle_result = json.loads(battle_result).get("api_data")
    

### 艦艇ドロップを確認

In [14]:
get_flag = battle_result.get("api_get_flag")

if get_flag[1] == 1:
    get_ship = battle_result.get("api_get_ship")
    print(f"「{get_ship.get("api_ship_name")}」をゲットしました")
else:
    print("艦艇ドロップはありませんでした")

艦艇ドロップはありませんでした


## 行き止まりか確認

In [15]:
if start_response.get("api_next") == 0:
    print("行き止まり")
else:
    print("行き止まりではありません")

行き止まりではありません


## api_req_map/next 戦闘継続

In [16]:
with open(root_dir + "/2025-01-30 10:35:54.851160_api_req_map-next.json") as f:
    next_response = f.read()
    next_response = json.loads(next_response).get("api_data")
    

### 羅針盤が表示されるか確認

In [17]:
print("羅針盤が表示される" if next_response.get("api_rashin_flg") else "羅針盤が表示されない")

羅針盤が表示される


### 次に訪れるセル

In [18]:
next_cell_id = next_response.get("api_no")

for d in cell_data:
    if d.get("api_no") == next_cell_id:
        next_cell_data = d
        break

print(f"次に訪れるセルのID: {next_cell_id}")
print(f"次に訪れるセルの情報: {next_cell_data}")
print(f"次のマスのイベント: {event_mapping.get(next_response.get("api_event_id"), "未知のイベント")}")

次に訪れるセルのID: 3
次に訪れるセルの情報: {'api_id': 3004, 'api_no': 3, 'api_color_no': 5, 'api_passed': 1}
次のマスのイベント: ボス戦闘


## api_req_sortie/battle 戦闘開始

In [19]:
with open(root_dir + "/2025-01-30 10:37:21.795726_api_req_sortie-battle.json") as f:
    battle_response = f.read()
    battle_response = json.loads(battle_response).get("api_data")
    

### 戦闘の情報

In [20]:
opening_taisen_flag = battle_response.get("api_opening_taik_flag")
opening_flag = battle_response.get("api_opening_flag")
hourai_flag = battle_response.get("api_hourai_flag")

print("先制対潜が発生する" if opening_taisen_flag else "先制対潜が発生しない")
print("先制雷撃が発生する" if opening_flag else "先制雷撃が発生しない")
print("砲撃戦１巡目が発生する" if hourai_flag[0] else "砲撃戦１巡目が発生しない")
print("砲撃戦２巡目が発生する" if hourai_flag[1] else "砲撃戦２巡目が発生しない")
print("砲撃戦３巡目が発生する" if hourai_flag[2] else "砲撃戦３巡目が発生しない")
print("雷撃戦が発生する" if hourai_flag[3] else "雷撃戦が発生しない")

先制対潜が発生しない
先制雷撃が発生しない
砲撃戦１巡目が発生する
砲撃戦２巡目が発生しない
砲撃戦３巡目が発生しない
雷撃戦が発生する


### 敵味方のHPを取得

In [21]:
friend_now_hp_list = battle_response.get("api_f_nowhps")
friend_max_hp_list = battle_response.get("api_f_maxhps")
enemy_now_hp_list = battle_response.get("api_e_nowhps")
enemy_max_hp_list = battle_response.get("api_e_maxhps")

for i, friend_now_hp in enumerate(friend_now_hp_list):
    print(f"味方{i+1}隻目のHP: {friend_now_hp}/{friend_max_hp_list[i]}")

for i, enemy_now_hp in enumerate(enemy_now_hp_list):
    print(f"敵{i+1}隻目のHP: {enemy_now_hp}/{enemy_max_hp_list[i]}")

味方1隻目のHP: 15/15
味方2隻目のHP: 13/13
味方3隻目のHP: 15/15
味方4隻目のHP: 14/15
味方5隻目のHP: 13/13
味方6隻目のHP: 16/16
敵1隻目のHP: 33/33
敵2隻目のHP: 20/20
敵3隻目のHP: 20/20


### 敵味方のダメージを算出

In [22]:

total_friend_damage_list = [0] * 7
total_enemy_damage_list = [0] * 7

if opening_taisen_flag:
    # TODO 先制対潜の情報を取得する
    pass
else:
    print("<先制対潜は発生しませんでした>")

if opening_flag:
    # TODO 先制雷撃の情報を取得する
    pass
else:
    print("<先制雷撃は発生しませんでした>")

# 砲撃戦の情報を取得する
for i in range(3):
    flag = hourai_flag[i]
    if not flag:
        break
    
    print("<砲撃戦" + str(i+1) + "巡目>")
    
    hougeki_data = battle_response.get(f"api_hougeki{i+1}")
    # 行動陣営フラグ 0=味方, 1=敵
    at_e_flag_list = hougeki_data.get("api_at_eflag")
    df_list = hougeki_data.get("api_df_list")
    damage_list = hougeki_data.get("api_damage")
    for i, at_e_flag in enumerate(at_e_flag_list):
        index = df_list[i][0]
        damage = damage_list[i][0]
        is_protected = damage % 1 == 0.1
        damage //= 1
        
        # ダメージを記録
        if at_e_flag:
            print(f"味方の{index + 1}隻目に{damage}ダメージ{"(旗艦を庇った)" if is_protected else ""}")
            total_friend_damage_list[index] += damage
        else:
            print(f"敵の{index + 1}隻目に{damage}ダメージ{"(旗艦を庇った)" if is_protected else ""}")
            total_enemy_damage_list[index] += damage

# 雷撃戦の情報を取得する
if hourai_flag[3]:
    raigeki = battle_response.get("api_raigeki")
    fdam = raigeki.get("api_fdam")
    edam = raigeki.get("api_edam")

    for i, f in enumerate(fdam):
        print(f"味方の{i+1}隻目に{f}ダメージ")
        total_friend_damage_list[i] += f

    for i, e in enumerate(edam):
        print(f"敵の{i+1}隻目に{e}ダメージ")
        total_enemy_damage_list[i] += e
else:
    print("<雷撃戦は発生しませんでした>")

print("<トータル>")
print(f"味方のダメージ合計: {total_friend_damage_list}")
print(f"敵のダメージ合計: {total_enemy_damage_list}")

<先制対潜は発生しませんでした>
<先制雷撃は発生しませんでした>
<砲撃戦1巡目>
敵の3隻目に19.0ダメージ
味方の5隻目に0ダメージ
敵の2隻目に10ダメージ
味方の5隻目に1ダメージ
敵の2隻目に0ダメージ
味方の2隻目に0ダメージ
敵の2隻目に8ダメージ
敵の3隻目に4ダメージ
敵の2隻目に6ダメージ
味方の1隻目に0ダメージ
味方の2隻目に8ダメージ
味方の3隻目に0ダメージ
味方の4隻目に0ダメージ
味方の5隻目に0ダメージ
味方の6隻目に0ダメージ
味方の7隻目に0ダメージ
敵の1隻目に23ダメージ
敵の2隻目に0ダメージ
敵の3隻目に0ダメージ
敵の4隻目に0ダメージ
敵の5隻目に0ダメージ
敵の6隻目に0ダメージ
敵の7隻目に0ダメージ
<トータル>
味方のダメージ合計: [0, 8, 0, 0, 1, 0, 0]
敵のダメージ合計: [23, 24, 23.0, 0, 0, 0, 0]


### 残存HPを算出 大破かどうかを算出

In [23]:
friend_rest_hp_list = [now - damage for now, damage in zip(friend_now_hp_list, total_friend_damage_list)]
enemy_rest_hp_list = [now - damage for now, damage in zip(enemy_now_hp_list, total_enemy_damage_list)]

for i, (rest, max) in enumerate(zip(friend_rest_hp_list, friend_max_hp_list)):
    huge_damage = rest <= max // 4
    print(f"味方{i+1}隻目の残りHP: {rest}/{max} {"(大破)" if huge_damage else ""}")

for i, (rest, max) in enumerate(zip(enemy_rest_hp_list, enemy_max_hp_list)):
    print(f"敵{i+1}隻目の残りHP: {rest}/{max}")

味方1隻目の残りHP: 15/15 
味方2隻目の残りHP: 5/13 
味方3隻目の残りHP: 15/15 
味方4隻目の残りHP: 14/15 
味方5隻目の残りHP: 12/13 
味方6隻目の残りHP: 16/16 
敵1隻目の残りHP: 10/33
敵2隻目の残りHP: -4/20
敵3隻目の残りHP: -3.0/20


### 夜戦を選択可能かどうか確認

In [24]:
can_midnight_battle = any([hp > 0 for hp in enemy_rest_hp_list])
print("夜戦を行える" if can_midnight_battle else "夜戦を行えない")

夜戦を行える


## api_req_battle_midnight/battle 夜戦

In [25]:
with open(root_dir + "/2025-01-30 10:38:46.735547_api_req_battle_midnight-battle.json") as f:
    midnight_battle_response = f.read()
    midnight_battle_response = json.loads(midnight_battle_response).get("api_data")